In [1]:
pip install kaggle


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:

pip install opendatasets


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:

import opendatasets as od
import pandas

In [5]:
od.download(
    "https://www.kaggle.com/datasets/erinkhoo/property-price-register-ireland/data")

2023-10-24 11:20:17,684 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x11e5053f0>: Failed to resolve 'www.kaggle.com' ([Errno 8] nodename nor servname provided, or not known)")': /api/v1/datasets/download/erinkhoo/property-price-register-ireland?datasetVersionNumber=None
2023-10-24 11:20:28,141 WARNING Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x11e505570>: Failed to resolve 'www.kaggle.com' ([Errno 8] nodename nor servname provided, or not known)")': /api/v1/datasets/download/erinkhoo/property-price-register-ireland?datasetVersionNumber=None
2023-10-24 11:20:38,148 WARNING Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NameResolutionError("<urllib3.connection.HT

MaxRetryError: HTTPSConnectionPool(host='www.kaggle.com', port=443): Max retries exceeded with url: /api/v1/datasets/download/erinkhoo/property-price-register-ireland?datasetVersionNumber=None (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x11e505810>: Failed to resolve 'www.kaggle.com' ([Errno 8] nodename nor servname provided, or not known)"))